In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from rnn_example import RNNModel, generator

In [2]:
total_datas = pd.read_csv("data/movies.csv", header=0)
    
words_list = total_datas['review'].apply(lambda x: x.split())
words_freq = defaultdict(int)
for i in range(len(total_datas)):
    for word in words_list[i]:
        words_freq[word] += 1
    
datas = []
for i in range(len(total_datas)):
    data = []
    for word in words_list[i]:
        if words_freq[word] >= 30:
            data.append(word)
    datas.append(data)
total_datas['review'] = datas
    
total_words = set()
for i in range(len(datas)):
    total_words |= set(datas[i])
vocab_size = len(total_words)
words_idx = dict(zip(total_words, range(len(total_words))))
idx_words = dict(zip(range(len(total_words)), total_words))
    
positive_indexs = np.where(total_datas['target'].values == 1)[0]
negative_indexs = np.where(total_datas['target'].values == 0)[0]
    
train_ratio = 0.7
np.random.seed(0)
selected = np.random.choice(positive_indexs,
                            size=int(train_ratio * len(total_datas) / 2),
                            replace=False).tolist() + \
           np.random.choice(negative_indexs,
                            size=int(train_ratio * len(total_datas) / 2),
                            replace=False).tolist()
selected.sort()
indexs = np.zeros(len(total_datas), dtype=np.bool)
indexs[selected] = True
train_data = total_datas[indexs]
test_data = total_datas[~indexs]
    
dataset = train_data['review'].tolist()
data_lengths = np.array(list(map(len, dataset)))
    
test_dataset = test_data['review'].tolist()
test_data_lengths = np.array(list(map(len, test_dataset)))
batch_test_datas = generator(
    test_dataset, test_data['target'].values, test_data_lengths, 64, words_idx
    )
    
hidden_size = 128

In [3]:
rnn = RNNModel(2, vocab_size, hidden_size)

/home/ubuntu/workspaces/py3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


## Loading model checkpoint ./rnn_example/model/model-24660 ...
INFO:tensorflow:Restoring parameters from ./rnn_example/model/model-24660


In [4]:
batch_data, batch_target, batch_length = batch_test_datas[0]
preds = rnn.predict(batch_data, batch_length)

In [5]:
for data, label, length, pred in zip(batch_data[:5], batch_target[:5], batch_length[:5], preds[:5]):
    review = []
    for i in range(length):
        review.append(idx_words[data[i]])
    review_str = ' '.join(review)
    print(f'review: {review_str}\n label: {label}, pred: {pred}')

review: even standards movie making setup cardboard lover presents dated extreme one half pair husband wife ex husband ex wife get back threat marriage another divorce eventual separation means jealousy schemes done much better classics girl friday philadelphia story movies features women strong screen presence played independent feminist characters movies women estranged divorced witty first husbands set marry men exact opposite would soon husbands passion plot cardboard lover switches gender norma cary grant role time ward ex boyfriend george sanders means hiring robert taylor pose problem much old playing role suited actress mid late twenties sanders involved piece furniture man would love fianc e seeing strange man come bathroom happens would knock lights cause huge scene robert taylor plays part trying channel cary grant half time speech overall essence worst part actress used parts gave sense intellectual dramatic presence playing seems put complete acting still part style acting